In [1]:
from splinter import Browser

browser = Browser('chrome')
url = "https://zh-tw.facebook.com/login/"
browser.visit(url)
username =input('username')
password =input('password')
search_input = browser.find_by_xpath('//*[@id="email"]')[0]
search_input.fill(username)
search_input = browser.find_by_xpath('//*[@id="pass"]')[0]
search_input.fill(password)
browser.find_by_xpath('//*[@id="loginbutton"]').first.click()
phonecheck = input('phonecheck_code')
search_input = browser.find_by_xpath('//*[@id="approvals_code"]')[0]
search_input.fill(phonecheck)
browser.find_by_xpath('//*[@id="checkpointSubmitButton"]').first.click()
browser.find_by_xpath('//*[@id="u_0_0"]/div/div[2]/ul/li[2]/div/label/strong').first.click()
browser.find_by_xpath('//*[@id="checkpointSubmitButton"]').first.click()

phonecheck_code916081


In [172]:
import time
from bs4 import BeautifulSoup
import re

success = 0
count = 0

while (success == 0 and count<4) :
    try:
        url = 'https://www.facebook.com/sunglin.chan'
        browser.visit(url)
        browser.find_by_text('More').first.click()
        browser.find_by_text('Likes').first.click()
        soup = BeautifulSoup(browser.html, 'lxml')        
        likes = soup.select('#pagelet_timeline_medley_likes > div._3cz > div.clearfix._1_ca > div._3dc.lfloat._ohe._5brz > a')[0].text.split()[1]
        numbers_of_likes = ''.join(re.findall('[\d]+',likes))
        a = int((int(numbers_of_likes.replace(',','')) - 20)/6)+5
        while a:
            browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
            time.sleep(0.2)
            a -= 1
        success = 1
    except:
        print('try',count,'times')
        count += 1
        

In [173]:
soup = BeautifulSoup(browser.html, 'lxml')

In [174]:
chose_link = 2
for link in soup.select('#collection_wrapper_2409997254')[0].select('div')[0].select('a'):
    if chose_link%3 == 0:
        print(link.text)
        print(link['href'])
    chose_link += 1
# soup.select('#collection_wrapper_2409997254')[0].select('div')[0].select('a')[1].text

Corey Schafer
https://www.facebook.com/CoreyMSchafer/
LiveCoding.tw
https://www.facebook.com/LiveCoding.tw/
區塊客 blockcast.it
https://www.facebook.com/blockcast.it/
R-J 革勢資本
https://www.facebook.com/RJTMESIS/
BondQ / MissQ
https://www.facebook.com/bondquote/
沒一村前端筆記
https://www.facebook.com/noootownnotes/
日月潭楓香咖啡莊園粉絲團
https://www.facebook.com/日月潭楓香咖啡莊園粉絲團-899486693453834/
Jenny's Cooking 珍妮烘焙手作
https://www.facebook.com/Jennygoodtimes/
寫點科普，請給指教。
https://www.facebook.com/ExpandeScientia/
在荷蘭工作的一百個理由
https://www.facebook.com/Eurosophy/
Learning By Hacking
https://www.facebook.com/datasci.info/
Oranje Express 荷事生非
https://www.facebook.com/OranjeExpress.org/
DTSO Delft Taiwanese Student Organisation
https://www.facebook.com/tudelftdtso/
德芳茶業
https://www.facebook.com/tehhongtea/
台灣荷蘭教育推廣協會       留學荷蘭問答區
https://www.facebook.com/Holland.Education.Association.Taiwan/
程式交易Alex Huang
https://www.facebook.com/AlgorithmTrading/
轉角國際 udn Global
https://www.facebook.com/udnglobal/
GPU PSC Taiwan
htt